# CourtRankRL Embedding Generation - Sentence Transformers (RTX 5090 GPU)

## Specifikáció
- **Modell**: google/embeddinggemma-300m via Sentence Transformers (>=5.1.0)
- **Input**: chunks.jsonl (processed court decisions)
- **Output**: embeddings.npy (float32, L2-normalized) és embedding_chunk_ids.json
- **Környezet**: RunPod RTX 5090 GPU (24GB VRAM)
- **Batch size**: 512 (GPU optimalizált)
- **Kritikus**: CSAK Sentence Transformers - AutoModel ZERO VECTOR-t produkál!

## Prompt-ok (automatikus Sentence Transformers-ben)
- **Document chunks**: `prompt_name="document"` → "title: none | text: {chunk_text}"
- **Query**: `prompt_name="query"` → "task: search result | query: {query_text}"
- **Normalization**: `normalize_embeddings=True` (automatikus L2-normalizálás)

## Memória kezelés
- Shard-okba írás nagy dataset-ekhez
- Konszolidáció a végén
- GPU memória monitoring

In [ ]:
# KRITIKUS: Csak Sentence Transformers - AutoModel ZERO VECTOR-t produkál!
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import os
import time
from pathlib import Path
import psutil
from typing import List
import torch

# --- Konfiguráció ---
BASE_PATH = Path("/workspace")
CHUNKS_PATH = BASE_PATH / "chunks.jsonl"
EMBEDDINGS_PATH = BASE_PATH / "embeddings.npy"
CHUNK_IDS_PATH = BASE_PATH / "embedding_chunk_ids.json"

# GPU konfiguráció RTX 5090-hez
BATCH_SIZE = 512
MAX_SEQ_LENGTH = 2048  # EmbeddingGemma default
SHARD_SIZE = 100_000  # Shard-okba írás

# HF token (környezeti változóból)
HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    raise ValueError("❌ HF_TOKEN környezeti változó hiányzik!")

print(f"🚀 RTX 5090 Embedding Generation indul...")
print(f"📂 Base path: {BASE_PATH}")
print(f"📄 Input chunks: {CHUNKS_PATH}")
print(f"📄 Output embeddings: {EMBEDDINGS_PATH}")
print(f"📄 Output chunk IDs: {CHUNK_IDS_PATH}")

# --- Chunks betöltés (pandas optimalizált - agents.md szerint) ---
def load_chunks(chunks_path: Path) -> List[dict]:
    """
    Chunks betöltése JSONL-ből pandas segítségével.
    
    pandas.read_json() 10-30x gyorsabb mint kézi json.loads() parsing.
    """
    import pandas as pd
    
    print(f"📥 Chunks betöltése: {chunks_path}")
    
    try:
        # pandas.read_json() C-optimalizált, sokkal gyorsabb
        df_chunks = pd.read_json(chunks_path, lines=True, encoding='utf-8')
        chunks = df_chunks.to_dict('records')
        print(f"✅ {len(chunks):,} chunks betöltve")
    except (ValueError, FileNotFoundError) as e:
        print(f"❌ Hiba a chunks betöltése során: {e}")
        chunks = []
    
    return chunks

chunks = load_chunks(CHUNKS_PATH)
chunk_texts = [chunk['text'] for chunk in chunks]
chunk_ids = [chunk['chunk_id'] for chunk in chunks]

print(f"📊 Összes chunk: {len(chunk_texts):,}")
print(f"💾 Memória használat: {psutil.virtual_memory().used / 1024**3:.1f}GB")

# --- Modell betöltés (CSAK Sentence Transformers!) ---
print("🤖 EmbeddingGemma modell betöltése (Sentence Transformers)...")

try:
    model = SentenceTransformer(
        "google/embeddinggemma-300m",
        token=HF_TOKEN,
        trust_remote_code=True,
        cache_dir="/tmp/hf_cache"
    )
    print("✅ Modell betöltve (Sentence Transformers)")
except Exception as e:
    print(f"❌ Modell betöltési hiba: {e}")
    raise

# GPU beállítások
if torch.cuda.is_available():
    device = torch.device("cuda")
    model = model.to(device)
    print(f"✅ GPU elérhető: {torch.cuda.get_device_name()}")
    print(f"💾 GPU memória: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
else:
    device = torch.device("cpu")
    print("⚠️ GPU nem elérhető - CPU használata (lassú!)")

# --- Embedding generálás ---
def generate_embeddings(model, texts: List[str], batch_size: int, device) -> np.ndarray:
    """Batch-es embedding generálás"""
    print(f"🔄 Embedding generálás indul: {len(texts):,} texts")
    
    all_embeddings = []
    start_time = time.time()
    
    # Batch-ek feldolgozása
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_start = time.time()
        
        try:
            # KRITIKUS: Sentence Transformers encode() használata
            # Automatikus prompt: "title: none | text: {chunk}"
            # Automatikus L2-normalizálás
            batch_embeddings = model.encode(
                batch_texts,
                prompt_name="document",  # Document prompt
                normalize_embeddings=True,  # L2-normalizálás
                batch_size=batch_size,
                convert_to_numpy=True,
                device=device,
                show_progress_bar=True
                # dtype=torch.float32 ELTÁVOLÍTVA - nem támogatott paraméter!
            )
            
            all_embeddings.append(batch_embeddings)
            
            # GPU memória monitoring
            if device.type == 'cuda':
                mem_used = torch.cuda.memory_allocated() / 1024**3
                print(f"  📊 Batch {i//batch_size + 1} kész | GPU memória: {mem_used:.1f}GB")
            
            batch_time = time.time() - batch_start
            print(f"  ⏱️ Batch idő: {batch_time:.2f}s")
            
        except Exception as e:
            print(f"❌ Batch hiba {i//batch_size + 1}-nál: {e}")
            raise
    
    # Konszolidáció
    embeddings = np.vstack(all_embeddings)
    total_time = time.time() - start_time
    
    print(f"✅ Embedding generálás kész: {embeddings.shape}")
    print(f"⏱️ Teljes idő: {total_time:.2f}s")
    print(f"⚡ Sebesség: {len(texts)/total_time:.1f} texts/sec")
    
    return embeddings

# Sanity check: Ellenőrizzük hogy nincsenek zero vector-ok
def validate_embeddings(embeddings: np.ndarray, chunk_ids: List[str]) -> tuple:
    """Embedding validáció és tisztítás"""
    print("🔍 Embedding validáció...")
    
    # NaN/Inf ellenőrzés
    finite_mask = np.isfinite(embeddings).all(axis=1)
    print(f"  📊 Finite embeddings: {finite_mask.sum():,}/{len(embeddings):,}")
    
    # Zero norma ellenőrzés
    norms = np.linalg.norm(embeddings, axis=1)
    nonzero_mask = norms > 1e-6
    print(f"  📊 Non-zero norm embeddings: {nonzero_mask.sum():,}/{len(embeddings):,}")
    
    # Kombinált maszk
    valid_mask = finite_mask & nonzero_mask
    num_invalid = (~valid_mask).sum()
    
    if num_invalid > 0:
        print(f"⚠️ {num_invalid:,} invalid embedding kiszűrve")
        embeddings = embeddings[valid_mask]
        chunk_ids = [cid for cid, keep in zip(chunk_ids, valid_mask) if keep]
    else:
        print("✅ Minden embedding valid")
    
    return embeddings, chunk_ids

# Generálás
embeddings = generate_embeddings(model, chunk_texts, BATCH_SIZE, device)

# Validáció és tisztítás
embeddings, chunk_ids = validate_embeddings(embeddings, chunk_ids)

# Végső dtype biztosítás
if embeddings.dtype != np.float32:
    embeddings = embeddings.astype(np.float32)

print(f"🎯 Végső alak: {embeddings.shape}")
print(f"💾 Memória használat: {embeddings.nbytes / 1024**3:.2f}GB")

# --- Mentés shard-okba (nagy dataset-ekhez) ---
def save_sharded_embeddings(embeddings: np.ndarray, chunk_ids: List[str], 
                           shard_size: int, output_path: Path, ids_path: Path):
    """Shard-okba mentés memória optimalizáláshoz"""
    print(f"💾 Shard-okba mentés (shard size: {shard_size:,})...")
    
    os.makedirs(output_path.parent, exist_ok=True)
    os.makedirs(ids_path.parent, exist_ok=True)
    
    # Shard-ok létrehozása
    for i in range(0, len(embeddings), shard_size):
        shard_idx = i // shard_size
        end_idx = min(i + shard_size, len(embeddings))
        
        shard_embeddings = embeddings[i:end_idx]
        shard_ids = chunk_ids[i:end_idx]
        
        shard_path = output_path.parent / f"embeddings_shard_{shard_idx}.npy"
        ids_shard_path = output_path.parent / f"chunk_ids_shard_{shard_idx}.json"
        
        np.save(shard_path, shard_embeddings)
        with open(ids_shard_path, 'w', encoding='utf-8') as f:
            json.dump(shard_ids, f, ensure_ascii=False, indent=2)
        
        print(f"  💾 Shard {shard_idx} mentve: {len(shard_embeddings):,} embeddings")
    
    # Konszolidált fájlok létrehozása
    print("🔄 Konszolidáció shard-okból...")
    
    # Embeddings összevonása
    consolidated_embeddings = []
    consolidated_ids = []
    
    shard_idx = 0
    while True:
        shard_path = output_path.parent / f"embeddings_shard_{shard_idx}.npy"
        ids_shard_path = output_path.parent / f"chunk_ids_shard_{shard_idx}.json"
        
        if not shard_path.exists():
            break
            
        shard_emb = np.load(shard_path)
        with open(ids_shard_path, 'r', encoding='utf-8') as f:
            shard_ids = json.load(f)
        
        consolidated_embeddings.append(shard_emb)
        consolidated_ids.extend(shard_ids)
        
        # Tisztítás
        os.remove(shard_path)
        os.remove(ids_shard_path)
        
        shard_idx += 1
    
    # Végső konszolidáció
    final_embeddings = np.vstack(consolidated_embeddings)
    print(f"✅ Konszolidált: {final_embeddings.shape}")
    
    # Mentés
    np.save(output_path, final_embeddings)
    with open(ids_path, 'w', encoding='utf-8') as f:
        json.dump(consolidated_ids, f, ensure_ascii=False, indent=2)
    
    print(f"💾 Konszolidált fájlok mentve")

# Mentés
save_sharded_embeddings(embeddings, chunk_ids, SHARD_SIZE, EMBEDDINGS_PATH, CHUNK_IDS_PATH)

# --- Végső validáció ---
print("🔍 Végső validáció...")

# Betöltés ellenőrzés
loaded_embeddings = np.load(EMBEDDINGS_PATH)
with open(CHUNK_IDS_PATH, 'r', encoding='utf-8') as f:
    loaded_ids = json.load(f)

# Ellenőrzések
assert loaded_embeddings.shape[0] == len(loaded_ids), "ID és embedding szám nem egyezik"
assert loaded_embeddings.shape[1] == 768, f"Embedding dim nem 768: {loaded_embeddings.shape[1]}"
assert loaded_embeddings.dtype == np.float32, f"Dtype nem float32: {loaded_embeddings.dtype}"

# NaN/Inf ellenőrzés
assert np.isfinite(loaded_embeddings).all(), "NaN/Inf az embeddingekben!"
assert np.all(np.linalg.norm(loaded_embeddings, axis=1) > 0), "Zero-norm embeddings!"

print("✅ Végső validáció sikeres")
print(f"📊 Végső statisztikák:")
print(f"  • Embeddings: {loaded_embeddings.shape}")
print(f"  • Chunk IDs: {len(loaded_ids)}")
print(f"  • Memória: {loaded_embeddings.nbytes / 1024**3:.2f}GB")

# --- Összefoglaló ---
print("="*80)
print("🎉 EMBEDDING GENERÁLÁS SIKERES!")
print("="*80)
print(f"📄 Kimeneti fájlok:")
print(f"  • {EMBEDDINGS_PATH}")
print(f"  • {CHUNK_IDS_PATH}")
print(f"📊 Vektorok: {loaded_embeddings.shape[0]:,}")
print(f"🎯 Dimenzió: {loaded_embeddings.shape[1]}")
print(f"💾 Fájlméret: {EMBEDDINGS_PATH.stat().st_size / 1024**3:.2f}GB")
print(f"⏱️ Futtatási idő: {time.time() - time.time():.2f}s")  # TODO: Track total time

print("
🚀 Következő lépés: Töltsd le az artifact-okat és futtasd a faiss_index_builder.ipynb-t")
print("💡 Tipp: Ellenőrizd hogy minden embedding L2-normalizált és finite!")

## Használat



## Kritikus megjegyzések

1. **CSAK Sentence Transformers!** AutoModel használata zero-vector-t produkál
2. **normalize_embeddings=True** kötelező az L2-normalizáláshoz
3. **float32** használata - EmbeddingGemma nem támogatja float16-ot
4. **prompt_name="document"** automatikusan hozzáadja a megfelelő prompt-ot
5. **Validáció** minden embedding-re - zero vector-ok azonnali detektálása
6. **dtype paraméter ELTÁVOLÍTVA** - Sentence Transformers nem támogatja ezt a paramétert